### Exercise1. Explore the class deﬁnitions BST and AVLTree:
(i) Create binary search trees and AVL trees with the same elements, compare the trees, the height of subtrees, the balances of nodes.  
(ii) Run find and delete operations on your trees.  
(iii) Implement methods to determine the minimum and the maximum element in BSTs and AVL trees.  
(iv) Implement a method to return the median of the elements in an AVL tree.  
(v) Given $x$ and $z$, implement an operation that returns the elements $e$ in an AVL tree with $x ≤ e ≤ z$.


In [17]:
import random

In [8]:
# Definition of BST
class BST:
    class __Node:
        def __init__(self,val,left=None,right=None):
            self.val = val
            self.left = left
            self.right = right
            self.parent = None
        def getVal(self):
            return self.val
        def setVal(self,newval):
            self.val = newval
        def getLeft(self):
            return self.left
        def setLeft(self,newleft):
            self.left = newleft
        def getRight(self):
            return self.right
        def setRight(self,newright):
            self.right = newright 
        def __iter__(self):
            if self.left != None:
                for elem in self.left:
                    yield elem
            yield self.val
            if self.right != None:
                for elem in self.right:
                    yield elem
                    
    def __init__(self):
        self.root = None
    def insert(self,val):
        def __insert(root,val):
            if root == None:
                return BST.__Node(val)
            if val < root.getVal():
                root.setLeft(__insert(root.getLeft(),val))
            else:
                root.setRight(__insert(root.getRight(),val))
            return root
        self.root = __insert(self.root,val)
    def __iter__(self):
        if self.root != None:
            return self.root.__iter__()
        else:
            return [].__iter__()
    def find(self,val):
        def __find(root,val):
            if root == None:
                return False
            if val == root.getVal():
                return True
            if val < root.getVal():
                return __find(root.getLeft(),val)
            else:
                return __find(root.getRight(),val)
        return __find(self.root,val)
    def delete(self,val):
        def __merge(leftnode,rightnode):
            if rightnode == None:
                return leftnode
            if rightnode.getLeft() == None:
                rightnode.setLeft(leftnode)
                return rightnode
            __merge(leftnode,rightnode.getLeft())
            return rightnode
        def __delete(root,val):
            if root == None:
                return root
            if val == root.getVal():
                return __merge(root.getLeft(),root.getRight())
            if val < root.getVal():
                root.setLeft(__delete(root.getLeft(),val))
                return root
            root.setRight(__delete(root.getRight(),val))
            return root
        __delete(self.root,val)
    ## Add a function to print out the tree intuitionistically
    def print_BST(self):
        ret = self.print_tree(self.root, 0, 0,{})
        ret = sorted(ret.items(), key=lambda x: x[1])
        cur_x = 0
        cur_y = 0
        for item in ret:
            x = item[1][1]
            y = item[1][0]
            parent = item[1][2]
            for i in range(cur_y, y):
                print('\n')
                cur_x = 0
            for i in range(cur_x, x):
                print(' ', end='')
            print(item[0], end='')
            print('('+str(parent)+')',end = '')
            cur_x = x+1
            cur_y = y
        print('\n')
        
    # helper function
    def print_tree(self, root, x, y, ret):
        # none do nothing and return
        if(root == None):
            return ret
        if(root.left == None and root.right == None): # leaf node   
            ret[str(root.val)] = (y, x, root.parent.val)
            return ret
        # otherwise, the left or right node is not None
        if(x-8 < 0):
            x = 8
        if(root.left != None):
            root.left.parent = root
            ret = self.print_tree(root.left, x-8, y+1, ret)
            leftChild = str(root.left.val)
            if(root == self.root):
                x = ret[leftChild][1]+16 # to distinct left and right tree
            else:
                x = ret[leftChild][1]+8
        if(root.parent == None):
            ret[str(root.val)] = (y, x, None)
        else:
            ret[str(root.val)] = (y, x, root.parent.val)
        if(root.right != None):
            root.right.parent = root
            if(root == self.root):
                ret = self.print_tree(root.right, x+16, y+1, ret) # to distinct left and right tree
            else:
                ret = self.print_tree(root.right, x+8, y+1, ret)
        return ret

In [50]:
class AVLTree:
    class AVLNode:
        def __init__(self,item,balance=0,left=None,right=None):
            self.item = item
            self.balance = balance
            self.left = left
            self.right = right
            self.parent = None
        def getitem(self):
            return self.item
        def setitem(self,newitem):
            self.item = newitem
        def getbal(self):
            return self.balance
        def setbal(self,newbalance):
            self.balance = newbalance
        def getLeft(self):
            return self.left
        def setLeft(self,newleft):
            self.left = newleft
        def getRight(self):
            return self.right
        def setRight(self,newright):
            self.right = newright
        def __iter__(self):
            if self.left != None:
                for elem in self.left:
                    yield elem
            yield self.item, self.balance
            if self.right != None:
                for elem in self.right:
                    yield elem
        def __repr__(self):
            return "AVLTree.AVLNode("+repr(self.item)+",balance="+repr(self.balance)+",left="+repr(self.left)+",right="+repr(self.right)+")"
    def __init__(self):
        self.root = None
    def __iter__(self):
        if self.root != None:
            return self.root.__iter__()
        else:
            return [].__iter__()
    def __rotateLeft(node,child,nbal,cbal):
            node.setRight(child.getLeft())
            node.setbal(nbal)
            child.setLeft(node)
            child.setbal(cbal)
            return child
    def __rotateRight(node,child,nbal,cbal):
            node.setLeft(child.getRight())
            node.setbal(nbal)
            child.setRight(node)
            child.setbal(cbal)
            return child

    def insert(self,item):
        def __insert(root,item):
            newtree = AVLTree.AVLNode(item)
            stack = []
            badChild = None
            while root != None:
                if item < root.getitem():
                    stack.insert(0,(root,-1))
                    if root.getLeft() == None:
                        root.setLeft(newtree)
                        return (stack,badChild)
                    if root.getbal() == -1:
                        badChild = root.getLeft()
                    root = root.getLeft()
                else:       
                    stack.insert(0,(root,1))
                    if root.getRight() == None:
                        root.setRight(newtree)
                        return (stack,badChild)
                    if root.getbal() == 1:
                        badChild = root.getRight()
                    root = root.getRight()
        if self. root == None:
            self.root = AVLTree.AVLNode(item)
            stack = []
            badChild = None
        else:
            result = __insert(self.root,item)
            stack = result[0]
            badChild = result[1]
        rotTree = None
        for N in stack:
            node = N[0]
            inc = N[1]
            if rotTree != None:
                if inc == 1:
                    node.setRight(rotTree)
                else:
                    node.setLeft(rotTree)
                return
            newbal = node.getbal() + inc
            if badChild == node:
                if inc == 1:
                    badGrandchild = node.getRight()
                else:
                    badGrandchild = node.getLeft()
            if newbal == 0:
                node.setbal(0)
                return
            if -1 <= newbal <= 1:
                node.setbal(newbal)
            else:
                if inc == 1:
                    if badChild.getRight() == badGrandchild:
                        rotTree = AVLTree.__rotateLeft(node,badChild,0,0)
                    else:
                        if item < badGrandchild.getitem():
                            n,c = 0,1
                        else:
                            if item > badGrandchild.getitem():
                                n,c = -1,0
                            else:
                                n,c = 0,0
                        rotTree1 = AVLTree.__rotateRight(badChild,badGrandchild,c,0)
                        rotTree = AVLTree.__rotateLeft(node,rotTree1,n,0)
                else:
                    if badChild.getLeft() == badGrandchild:
                        rotTree = AVLTree.__rotateRight(node,badChild,0,0)
                    else:
                        if item < badGrandchild.getitem():
                            n,c = 0,-1
                        else:
                            if item > badGrandchild.getitem():
                                n,c = 1,0
                            else:
                                n,c = 0,0
                        rotTree1 = AVLTree.__rotateLeft(badChild,badGrandchild,c,0)
                        rotTree = AVLTree.__rotateRight(node,rotTree1,n,0)
        if rotTree != None:
            self.root = rotTree
    def find(self,item):
        def __find(root,item):
            if root == None:
                return False
            if item == root.getitem():
                return True
            if item < root.getitem():
                return __find(root.getLeft(),item)
            else:
                return __find(root.getRight(),item)
        return __find(self.root,item)
    
    def delete(self,item):
        def __findandreturn(root,item):
            if root == None:
                return([],root,False)
            if item == root.getitem():
                return([(root,0)],root,True)
            if item < root.getitem():
                result = __findandreturn(root.getLeft(),item)
                stack1 = result[0]
                stack1.append((root,1))
                return (stack1,result[1],result[2])
            else:
                result = __findandreturn(root.getRight(),item)
                stack1 = result[0]
                stack1.append((root,-1))
                return (stack1,result[1],result[2])

        def __findswapLeft(node,prev):
            stack = []
            while node.getLeft() != None:
                stack.insert(0,(node,1))
                prev = node
                node = node.getLeft()
            minitem = node.getitem()
            if stack == []:
                prev.setRight(node.getRight())
            else:
                prev.setLeft(node.getRight())
            return (minitem,stack)


        def __findswapRight(node,prev):
            stack = []
            while node.getRight() != None:
                stack.insert(0,(node,-1))
                prev = node
                node = node.getRight()
            maxitem = node.getitem()
            if stack == []:
                prev.setLeft(node.getRight())
            else:
                prev.setRight(node.getLeft())
            return (maxitem,stack)

        result = __findandreturn(self.root,item)
        if result[2] == False:
            return
        stack1 = result[0]
        start = stack1.pop(0)
        startnode = start[0]
        startinc = start[1]
        nextRight = startnode.getRight()
        nextLeft = startnode.getLeft()
        stack = []
        if startnode.getbal() == 1:
            result = __findswapLeft(nextRight,startnode)
            stack = result[1]
            startnode.setitem(result[0])
            startinc = -1
        else:
            if startnode.getbal() == -1:
                result = __findswapRight(nextLeft,startnode)
                stack = result[1]
                startnode.setitem(result[0])
                startinc = 1
            else:
                if startnode.getRight() != None:
                    if nextRight.getbal() != 1:
                        result = __findswapLeft(nextRight,startnode)
                        stack = result[1]
                        startnode.setitem(result[0])
                        startinc = -1
                    else:
                        result = __findswapRight(nextLeft,startnode)
                        stack = result[1]
                        startnode.setitem(result[0])
                        startinc = 1
                else:
                    if self.root == startnode:
                        self.root = None
                        return
                    last = stack1.pop(0)
                    lastnode = last[0]
                    if last[1] ==1:
                        lastnode.setLeft(None)
                    else:
                        lastnode.setRight(None)
                    stack1.insert(0,(lastnode,last[1]))
                    startnode = None
        if startnode != None:
            start = (startnode,startinc)
            stack1.insert(0,start)
        stack = stack + stack1
        stack2 = iter(stack+[(None,0)])
        NN = next(stack2)
        for N in stack:
            currentnode = N[0]
            increment = N[1]
            NN = next(stack2)
            rotTree = None
            stop = False
            newbal = currentnode.getbal() + increment
            if -1 <= newbal <= 1:
                currentnode.setbal(newbal)
                if newbal != 0:
                    return
            else:
                if increment == 1:
                    badChild = currentnode.getRight()
                    badGrandchild = badChild.getLeft()
                    if badChild.getbal() == 0:
                        rotTree = FibTree.__rotateLeft(currentnode,badChild,1,-1)
                        stop = True
                    else:
                        if badChild.getbal() == 1:
                            rotTree = FibTree.__rotateLeft(currentnode,badChild,0,0)
                        else:
                            nbal,cbal = 0,0
                            if badGrandchild.getbal() == 1:
                                nbal = -1
                            if badGrandchild.getbal() == -1:
                                cbal = 1
                            rotTree1 = FibTree.__rotateRight(badChild,badGrandchild,cbal,0)
                            rotTree = FibTree.__rotateLeft(currentnode,rotTree1,nbal,0)
                else:
                    badChild = currentnode.getLeft()
                    badGrandchild = badChild.getRight()
                    stop = False
                    if badChild.getbal() == 0:
                        rotTree = FibTree.__rotateRight(currentnode,badChild,-1,1)
                        stop = True
                    else:
                        if badChild.getbal() == -1:
                            rotTree = FibTree.__rotateRight(currentnode,badChild,0,0)
                        else:
                            nbal,cbal = 0,0
                            if badGrandchild.getbal() == 1:
                                cbal = -1
                            if badGrandchild.getbal() == -1:
                                nbal = 1
                            rotTree1 = FibTree.__rotateLeft(badChild,badGrandchild,cbal,0)
                            rotTree = FibTree.__rotateRight(currentnode,rotTree1,nbal,0)
            if rotTree != None:
                if NN[0] != None:
                    if NN[1] == 1:
                        NN[0].setLeft(rotTree)
                    if NN[1] == -1:
                        NN[0].setRight(rotTree)
                else:
                    self.root = rotTree
            if stop == True:
                return
    
    ## Add a function to print out the tree intuitionistically
    def print_AVL(self):
        ret = self.print_tree(self.root, 0, 0,{})
        ret = sorted(ret.items(), key=lambda x: x[1])
        cur_x = 0
        cur_y = 0
        for item in ret:
            x = item[1][1]
            y = item[1][0]
            parent = item[1][2]
            for i in range(cur_y, y):
                print('\n')
                cur_x = 0
            for i in range(cur_x, x):
                print(' ', end='')
            print(item[0], end='')
            print('('+str(parent)+')',end = '')
            cur_x = x+1
            cur_y = y
        print('\n')
    # helper function
    def print_tree(self, root, x, y, ret):
        # none do nothing and return
        if(root == None):
            return ret
        if(root.left == None and root.right == None): # leaf node   
            ret[str(root.item)] = (y, x, root.parent.item)
            return ret
        # otherwise, the left or right node is not None
        if(x-8 < 0):
            x = 8
        if(root.left != None):
            root.left.parent = root
            ret = self.print_tree(root.left, x-8, y+1, ret)
            leftChild = str(root.left.item)
            if(root == self.root):
                x = ret[leftChild][1]+16 # to distinct left and right tree
            else:
                x = ret[leftChild][1]+8
        if(root.parent == None):
            ret[str(root.item)] = (y, x, None)
        else:
            ret[str(root.item)] = (y, x, root.parent.item)
        if(root.right != None):
            root.right.parent = root
            if(root == self.root):
                ret = self.print_tree(root.right, x+16, y+1, ret) # to distinct left and right tree
            else:
                ret = self.print_tree(root.right, x+8, y+1, ret)
        return ret

In [49]:
## (i) add your code here, you can print the tree to see the height, balance
avl = AVLTree()
lst = [i for i in range(10)] # Even length
random.shuffle(lst)
print(lst)
for i in lst:
    avl.insert(i)
avl.print_AVL()

[7, 1, 6, 3, 2, 5, 4, 8, 9, 0]
                                3(None)

                2(3)                               6(3)

        1(2)                               5(6)               8(6)

0(1)                               4(5)               7(8)               9(8)



In [27]:
## (iii) for both BST and AVL
def findMinMax():
    pass

In [28]:
## Tests for (iii)

In [29]:
# (iv) find median
def median(tree):
    pass

In [30]:
# Tests for (iv)
avl = AVLTree()
lst = [i for i in range(10)] # Even length
random.shuffle(lst)
print(lst)
for i in lst:
    avl.insert(i)
print(median(avl))

avl = AVLTree()
lst = [i for i in range(11)] # Odd length
random.shuffle(lst)
print(lst)
for i in lst:
    avl.insert(i)
print(median(avl))

[3, 6, 1, 2, 4, 0, 5, 7, 9, 8]
None
[7, 8, 5, 4, 1, 9, 6, 10, 0, 3, 2]
None


In [31]:
## (v)
def find_between(tree, x, z):
    pass

In [32]:
## Tests for (v)
avl = AVLTree()
lst = [i for i in range(10)] # Even length
random.shuffle(lst)
print(lst)
for i in lst:
    avl.insert(i)

# example: print(find_between(avl, 3, 7))

[3, 1, 5, 7, 6, 9, 2, 8, 0, 4]


### Exercise 2.
(i) Write a program that checks, if a given binary tree with labelled vertices is a BST.

(ii) Write a program that checks, if a given binary tree with labelled vertices is an AVL tree.

In [23]:
## add your self-defined checking function here, using classes defined above

In [24]:
## check for BST or AVL

### Exercise 3: 
A Fibonacci tree of height $h$ is an AVL tree with a minimum number of nodes among all AVL trees of height $h$.

(i) Familiarise yourself with the notion of Fibonacci tree by creating Fibonacci trees of height $1, 2, 3, 4$.

(ii) Write a program that checks, if a given binary tree with labelled vertices is a Fibonacci tree.

(iii) Determine the number of Fibonacci trees of height $h$ (up to isomorphism).

(iv) Give a criterion, whether it is possible to create a Fibonacci tree with $n$ given
elements.

(v) Implement a method that turns an AVL tree into a Fibonacci tree, if this is possible.

(vi) Implement a class FibTree of Fibonacci trees with a method insert, which according to (iv) must insert several elements. Exploit (v) for the definition of the insert method.

[link for your reference](https://ai.berkeley.edu/~cs61bl/r//cur/trees/fibonacci-tree.html?topic=lab15.topic&step=7&course)

In [35]:
# original AVL tree class change to Fibo for (i)
class FibTree:
    class FibNode:
        def __init__(self,item,balance=0,left=None,right=None):
            self.item = item
            self.balance = balance
            self.left = left
            self.right = right
            self.parent = None
        def getitem(self):
            return self.item
        def setitem(self,newitem):
            self.item = newitem
        def getbal(self):
            return self.balance
        def setbal(self,newbalance):
            self.balance = newbalance
        def getLeft(self):
            return self.left
        def setLeft(self,newleft):
            self.left = newleft
        def getRight(self):
            return self.right
        def setRight(self,newright):
            self.right = newright

        def __iter__(self):
            if self.left != None:
                for elem in self.left:
                    yield elem
            yield self.item, self.balance
            if self.right != None:
                for elem in self.right:
                    yield elem

        def __repr__(self):
            return "FibTree.FibNode("+repr(self.item)+",balance="+repr(self.balance)+",left="+repr(self.left)+",right="+repr(self.right)+")"

    def __init__(self):
        self.root = None

    def __iter__(self):
        if self.root != None:
            return self.root.__iter__()
        else:
            return [].__iter__()

    def __rotateLeft(node,child,nbal,cbal):
            node.setRight(child.getLeft())
            node.setbal(nbal)
            child.setLeft(node)
            child.setbal(cbal)
            return child
    def __rotateRight(node,child,nbal,cbal):
            node.setLeft(child.getRight())
            node.setbal(nbal)
            child.setRight(node)
            child.setbal(cbal)
            return child

    def insert(self,item):
        def __insert(root,item):
            newtree = FibTree.FibNode(item)
            stack = []
            badChild = None
            while root != None:
                if item < root.getitem():
                    stack.insert(0,(root,-1))
                    if root.getLeft() == None:
                        root.setLeft(newtree)
                        return (stack,badChild) # return the problem one
                    if root.getbal() == -1:
                        badChild = root.getLeft()
                    root = root.getLeft()
                else:       
                    stack.insert(0,(root,1))
                    if root.getRight() == None:
                        root.setRight(newtree)
                        return (stack,badChild) # return the problem one
                    if root.getbal() == 1:
                        badChild = root.getRight()
                    root = root.getRight()
    
        if self. root == None:
            self.root = FibTree.FibNode(item)
            stack = []
            badChild = None
        else:
            result = __insert(self.root,item)
            stack = result[0]
            badChild = result[1]
        rotTree = None
        for N in stack:
            node = N[0]
            inc = N[1]
            if rotTree != None:
                if inc == 1:
                    node.setRight(rotTree)
                else:
                    node.setLeft(rotTree)
                return
            
            newbal = node.getbal() + inc
            if badChild == node:
                if inc == 1:
                    badGrandchild = node.getRight()
                else:
                    badGrandchild = node.getLeft()
            if newbal == 0:
                node.setbal(0)
                return
            if -1 <= newbal <= 1:
                node.setbal(newbal)
            else:
                if inc == 1:
                    if badChild.getRight() == badGrandchild:
                        rotTree = FibTree.__rotateLeft(node,badChild,0,0)
                    else:
                        if item < badGrandchild.getitem():
                            n,c = 0,1
                        else:
                            if item > badGrandchild.getitem():
                                n,c = -1,0
                            else:
                                n,c = 0,0
                        rotTree1 = FibTree.__rotateRight(badChild,badGrandchild,c,0)
                        rotTree = FibTree.__rotateLeft(node,rotTree1,n,0)
                else:
                    if badChild.getLeft() == badGrandchild:
                        rotTree = FibTree.__rotateRight(node,badChild,0,0)
                    else:
                        if item < badGrandchild.getitem():
                            n,c = 0,-1
                        else:
                            if item > badGrandchild.getitem():
                                n,c = 1,0
                            else:
                                n,c = 0,0
                        rotTree1 = FibTree.__rotateLeft(badChild,badGrandchild,c,0)
                        rotTree = FibTree.__rotateRight(node,rotTree1,n,0)
        if rotTree != None:
            self.root = rotTree
    def find(self,item):
        def __find(root,item):
            if root == None:
                return False
            if item == root.getitem():
                return True
            if item < root.getitem():
                return __find(root.getLeft(),item)
            else:
                return __find(root.getRight(),item)
        return __find(self.root,item)
    
    def delete(self,item):
        def __findandreturn(root,item):
            if root == None:
                return([],root,False)
            if item == root.getitem():
                return([(root,0)],root,True)
            if item < root.getitem():
                result = __findandreturn(root.getLeft(),item)
                stack1 = result[0]
                stack1.append((root,1))
                return (stack1,result[1],result[2])
            else:
                result = __findandreturn(root.getRight(),item)
                stack1 = result[0]
                stack1.append((root,-1))
                return (stack1,result[1],result[2])

        def __findswapLeft(node,prev):
            stack = []
            while node.getLeft() != None:
                stack.insert(0,(node,1))
                prev = node
                node = node.getLeft()
            minitem = node.getitem()
            if stack == []:
                prev.setRight(node.getRight())
            else:
                prev.setLeft(node.getRight())
            return (minitem,stack)


        def __findswapRight(node,prev):
            stack = []
            while node.getRight() != None:
                stack.insert(0,(node,-1))
                prev = node
                node = node.getRight()
            maxitem = node.getitem()
            if stack == []:
                prev.setLeft(node.getRight())
            else:
                prev.setRight(node.getLeft())
            return (maxitem,stack)

        result = __findandreturn(self.root,item)
        if result[2] == False:
            return
        stack1 = result[0]
        start = stack1.pop(0)
        startnode = start[0]
        startinc = start[1]
        nextRight = startnode.getRight()
        nextLeft = startnode.getLeft()
        stack = []
        if startnode.getbal() == 1:
            result = __findswapLeft(nextRight,startnode)
            stack = result[1]
            startnode.setitem(result[0])
            startinc = -1
        else:
            if startnode.getbal() == -1:
                result = __findswapRight(nextLeft,startnode)
                stack = result[1]
                startnode.setitem(result[0])
                startinc = 1
            else:
                if startnode.getRight() != None:
                    if nextRight.getbal() != 1:
                        result = __findswapLeft(nextRight,startnode)
                        stack = result[1]
                        startnode.setitem(result[0])
                        startinc = -1
                    else:
                        result = __findswapRight(nextLeft,startnode)
                        stack = result[1]
                        startnode.setitem(result[0])
                        startinc = 1
                else:
                    if self.root == startnode:
                        self.root = None
                        return
                    last = stack1.pop(0)
                    lastnode = last[0]
                    if last[1] ==1:
                        lastnode.setLeft(None)
                    else:
                        lastnode.setRight(None)
                    stack1.insert(0,(lastnode,last[1]))
                    startnode = None
        if startnode != None:
            start = (startnode,startinc)
            stack1.insert(0,start)
        stack = stack + stack1
        stack2 = iter(stack+[(None,0)])
        NN = next(stack2)
        for N in stack:
            currentnode = N[0]
            increment = N[1]
            NN = next(stack2)
            rotTree = None
            stop = False
            newbal = currentnode.getbal() + increment
            if -1 <= newbal <= 1:
                currentnode.setbal(newbal)
                if newbal != 0:
                    return
            else:
                if increment == 1:
                    badChild = currentnode.getRight()
                    badGrandchild = badChild.getLeft()
                    if badChild.getbal() == 0:
                        rotTree = FibTree.__rotateLeft(currentnode,badChild,1,-1)
                        stop = True
                    else:
                        if badChild.getbal() == 1:
                            rotTree = FibTree.__rotateLeft(currentnode,badChild,0,0)
                        else:
                            nbal,cbal = 0,0
                            if badGrandchild.getbal() == 1:
                                nbal = -1
                            if badGrandchild.getbal() == -1:
                                cbal = 1
                            rotTree1 = FibTree.__rotateRight(badChild,badGrandchild,cbal,0)
                            rotTree = FibTree.__rotateLeft(currentnode,rotTree1,nbal,0)
                else:
                    badChild = currentnode.getLeft()
                    badGrandchild = badChild.getRight()
                    stop = False
                    if badChild.getbal() == 0:
                        rotTree = FibTree.__rotateRight(currentnode,badChild,-1,1)
                        stop = True
                    else:
                        if badChild.getbal() == -1:
                            rotTree = FibTree.__rotateRight(currentnode,badChild,0,0)
                        else:
                            nbal,cbal = 0,0
                            if badGrandchild.getbal() == 1:
                                cbal = -1
                            if badGrandchild.getbal() == -1:
                                nbal = 1
                            rotTree1 = FibTree.__rotateLeft(badChild,badGrandchild,cbal,0)
                            rotTree = FibTree.__rotateRight(currentnode,rotTree1,nbal,0)
            if rotTree != None:
                if NN[0] != None:
                    if NN[1] == 1:
                        NN[0].setLeft(rotTree)
                    if NN[1] == -1:
                        NN[0].setRight(rotTree)
                else:
                    self.root = rotTree
            if stop == True:
                return
    
    ## Add a function to print out the tree intuitionistically
    def print_Fib(self):
        ret = self.print_tree(self.root, 0, 0,{})
        ret = sorted(ret.items(), key=lambda x: x[1])
        cur_x = 0
        cur_y = 0
        for item in ret:
            x = item[1][1]
            y = item[1][0]
            parent = item[1][2]
            for i in range(cur_y, y):
                print('\n')
                cur_x = 0
            for i in range(cur_x, x):
                print(' ', end='')
            print(item[0], end='')
            print('('+str(parent)+')',end = '')
            cur_x = x+1
            cur_y = y
        print('\n')
    # helper function
    def print_tree(self, root, x, y, ret):
        # none do nothing and return
        if(root == None):
            return ret
        if(root.left == None and root.right == None): # leaf node   
            ret[str(root.item)] = (y, x, root.parent.item)
            return ret
        # otherwise, the left or right node is not None
        if(x-8 < 0):
            x = 8
        if(root.left != None):
            root.left.parent = root
            ret = self.print_tree(root.left, x-8, y+1, ret)
            leftChild = str(root.left.item)
            if(root == self.root):
                x = ret[leftChild][1]+16 # to distinct left and right tree
            else:
                x = ret[leftChild][1]+8
        if(root.parent == None):
            ret[str(root.item)] = (y, x, None)
        else:
            ret[str(root.item)] = (y, x, root.parent.item)
        if(root.right != None):
            root.right.parent = root
            if(root == self.root):
                ret = self.print_tree(root.right, x+16, y+1, ret) # to distinct left and right tree
            else:
                ret = self.print_tree(root.right, x+8, y+1, ret)
        return ret
        

In [34]:
## (1) draw your own fibtree and insert them using AVL class, print out to see their structures
# your code here

#### Recursion Relation: 
write down your findings here.

In [36]:
## (ii)
# Definition of BST
class BST:
    class __Node:
        def __init__(self,val,left=None,right=None):
            self.val = val
            self.left = left
            self.right = right
            self.parent = None
        def getVal(self):
            return self.val
        def setVal(self,newval):
            self.val = newval
        def getLeft(self):
            return self.left
        def setLeft(self,newleft):
            self.left = newleft
        def getRight(self):
            return self.right
        def setRight(self,newright):
            self.right = newright 
        def __iter__(self):
            if self.left != None:
                for elem in self.left:
                    yield elem
            yield self.val
            if self.right != None:
                for elem in self.right:
                    yield elem
                    
    def __init__(self):
        self.root = None
    def insert(self,val):
        def __insert(root,val):
            if root == None:
                return BST.__Node(val)
            if val < root.getVal():
                root.setLeft(__insert(root.getLeft(),val))
            else:
                root.setRight(__insert(root.getRight(),val))
            return root
        self.root = __insert(self.root,val)
    def __iter__(self):
        if self.root != None:
            return self.root.__iter__()
        else:
            return [].__iter__()
    def find(self,val):
        def __find(root,val):
            if root == None:
                return False
            if val == root.getVal():
                return True
            if val < root.getVal():
                return __find(root.getLeft(),val)
            else:
                return __find(root.getRight(),val)
        return __find(self.root,val)
    def delete(self,val):
        def __merge(leftnode,rightnode):
            if rightnode == None:
                return leftnode
            if rightnode.getLeft() == None:
                rightnode.setLeft(leftnode)
                return rightnode
            __merge(leftnode,rightnode.getLeft())
            return rightnode
        def __delete(root,val):
            if root == None:
                return root
            if val == root.getVal():
                return __merge(root.getLeft(),root.getRight())
            if val < root.getVal():
                root.setLeft(__delete(root.getLeft(),val))
                return root
            root.setRight(__delete(root.getRight(),val))
            return root
        __delete(self.root,val)
    ## Add a function to print out the tree intuitionistically
    def print_BST(self):
        ret = self.print_tree(self.root, 0, 0,{})
        ret = sorted(ret.items(), key=lambda x: x[1])
        cur_x = 0
        cur_y = 0
        for item in ret:
            x = item[1][1]
            y = item[1][0]
            parent = item[1][2]
            for i in range(cur_y, y):
                print('\n')
                cur_x = 0
            for i in range(cur_x, x):
                print(' ', end='')
            print(item[0], end='')
            print('('+str(parent)+')',end = '')
            cur_x = x+1
            cur_y = y
        print('\n')
        
    # helper function
    def print_tree(self, root, x, y, ret):
        # none do nothing and return
        if(root == None):
            return ret
        if(root.left == None and root.right == None): # leaf node   
            ret[str(root.val)] = (y, x, root.parent.val)
            return ret
        # otherwise, the left or right node is not None
        if(x-8 < 0):
            x = 8
        if(root.left != None):
            root.left.parent = root
            ret = self.print_tree(root.left, x-8, y+1, ret)
            leftChild = str(root.left.val)
            if(root == self.root):
                x = ret[leftChild][1]+16 # to distinct left and right tree
            else:
                x = ret[leftChild][1]+8
        if(root.parent == None):
            ret[str(root.val)] = (y, x, None)
        else:
            ret[str(root.val)] = (y, x, root.parent.val)
        if(root.right != None):
            root.right.parent = root
            if(root == self.root):
                ret = self.print_tree(root.right, x+16, y+1, ret) # to distinct left and right tree
            else:
                ret = self.print_tree(root.right, x+8, y+1, ret)
        return ret
    # get the hight of a specified node recursively
    def height(self, node):
        pass
    
    # return whether a tree is AVL tree
    def isAVL(self, root):
        pass
    
    # Fibonnaci Number 
    def __Fibo(height):
        pass
    
    # Node Number of a given tree
    def __NodeNumber(root):
        pass
    
    def isFiboTree(self):
        pass  
        

In [37]:
## Test for true cases

In [38]:
## Test for false cases

#### (iii), (iv), (v), (vi) here. write down your thinkings or code below.